# Android Dalvik虚拟机
　　参考：[Dalvik指令集](http://pallergabor.uw.hu/androidblog/dalvik_opcodes.html)<br>

## 一、简介
　　Dalvik是Google公司自己设计用于Android平台的虚拟机。它可以支持已转换为 .dex（即Dalvik Executable）格式的Java应用程序的运行，.dex格式是专为Dalvik设计的一种压缩格式，适合内存和处理器速度有限的系统。<br>
　　Dalvik 经过优化，允许在有限的内存中同时运行多个虚拟机的实例，并且每一个Dalvik 应用作为一个独立的Linux 进程执行。独立的进程可以防止在虚拟机崩溃的时候所有程序都被关闭。<br>
　　Dalvik虚拟机是基于寄存器架构的，数据的访问通过寄存器单间直接传递。对java的每个线程都有一个pc计数器和一个java栈。Pc计数器类似arm cpu中的PC寄存器和x86 cpu中的IP寄存器，不同的是，PC计数器只对当前方法有效。<br>
　　Dalvik和标准Java虚拟机(JVM)首要差别是：Dalvik 基于寄存器架构，而 JVM 基于栈。基于寄存器的虚拟机对于编译后变大的程序来说，在它们执行的时候，花费的时间更短。<br>
　　Dalvik和Java运行环境的区别：<br>
　　1、Dalvik主要是完成对象生命周期管理，堆栈管理，线程管理，安全和异常管理，以及垃圾回收等等重要功能。<br>
　　2、Dalvik负责进程隔离和线程管理，每一个Android应用在底层都会对应一个独立的Dalvik虚拟机实例，其代码在虚拟机的解释下得以执行。<br>
　　3、不同于Java虚拟机运行java字节码，Dalvik虚拟机运行的是其专有的文件格式Dex<br>
　　4、dex文件格式可以减少整体文件尺寸，提高I/o操作的类查找速度。<br>
　　5、odex是为了在运行过程中进一步提高性能，对dex文件的进一步优化。<br>
　　6、所有的Android应用的线程都对应一个Linux线程，虚拟机因而可以更多的依赖操作系统的线程调度和管理机制。<br>
　　7、有一个特殊的虚拟机进程Zygote，他是虚拟机实例的孵化器。它在系统启动的时候就会产生，它会完成虚拟机的初始化，库的加载，预制类库和初始化的操作。如果系统需要一个新的虚拟机实例，它会迅速复制自身，以最快的数据提供给系统。对于一些只读的系统库，所有虚拟机实例都和Zygote共享一块内存区域。<br>
　　**认识Dalvik 会有什么作用**<br>
　　我们一开始学习Android开发是用Java写的，运行安装在自己手机上，但随着app的开发，我们会感觉每天都在写着同样的代码，这时我们会渴望学习更深入的东西，会下载源代码看是如何实现的，比如有趣的是点击了桌面上的图标之后，打开了程序，原理是找到包名打开的:<br>
>Intent intent = new Intent();<br>
intent.setClassName(getApplicationContext(), "应用的包名");<br>
startActivity(intent);<br>

　　那么包名字是哪里来的呢，是因为桌面是一个系统程序·com.android.launcher 负责加载系统中安装的app。<br>
　　本章将介绍Dalvik虚拟机特性与基于Dalvik字节码的汇编语言知识。<br>
## 二、Dalvik虚拟机特点——Android程序运行原理

### 2.1、Dalvik 虚拟机概述

### 2.2、Dalvik虚拟机与Java虚拟机的区别
#### 2.2.1、Java虚拟机运行的是Java字节码，Dalvik虚拟机运行的是Dalvik字节码
#### 2.2.2、Dalvik的可执行文件体积更小
#### 2.2.3、Java虚拟机与Dalvik虚拟机架构不同

### 2.3、Dalvik虚拟机是如何执行程序的

### 2.4、关于Dalvik虚拟机 JIT（即时编译、动态编译）
　　JIT，是一种通过在运行时将字节码编译为机器码的技术，可以使程序执行速度更快。<br>
　　主流的JIT包含两种字节码编译方式：<br>
　　1、method方式：以函数或方法为单位进行编译。<br>
　　2、trace方式：以trace为单位进行编译。<br>
　　程序中，一般很少是顺序执行代码的，多数代码都分成了好几条执行路径，其中函数的有些路径在实际运行过程中是很少被执行的，这部分路径被称为“冷路径”，而执行比较频繁的路径被称为“热路径”。采用传统 method 方式，以函数/方法为粒度会编译整个方法的代码，这样会在冷路径上浪费很多编译时间， 并且耗费更多的内存空间，却没有得到执行速度的提升。trace 方式编译，则能够快速的获取“热路径”代码，使用更短的时间与更少的内存来编译代码，等真的执行到冷路径时跳回到解释器或其它备用实行方式继续。<br>
　　

## 三、Dalvik汇编基础
　　Dalvik虚拟机为自己专门设计了一套指令集，并制定了自己的指令格式和调用规范。我们将Dalvik指令集组成的代码称为Dalvik汇编代码，由Dalvik汇编代码构成的程序文件即为smali文件（以.smali结尾,一般反编译而来，并非程序员编写，但可编辑后再编译）。<br>
### 3.1、smali文件初识
#### 3.1.1、smali文件头说明
　　在smali 文件的头3 行描述了当前类的一些信息。<br>
　　smali文件的头3行:
>.class public Lcom/example/com/MainActivity;　# .class < 访问权限> \[ 修饰关键字\] < 类名><br>
.super Ljava/lang/Object;　　　# .super <父类名> <br> 
.source "MainActivity.java"　　　# .source <源文件名><br>

　　说明：<br>
　　1、指定了当前类的类名,访问权限为public，类名开头的L是遵循Dalvik字节码的相关约定（Java对象）。<br>
　　2、super指令指定了当前类的父类，父类为Object。<br>
　　3、指定了当前类的源文件名。<br>
　　**注意：** 经过混淆的dex文件，反编译出来的smali代码可能没有源文件信息，source可能为空。<br>

　　访问权限修饰符即所谓的public,protected,private，而非权限修饰符则指的是final,abstract,static，两者都可以为空。
举例如下：<br>
>.class public final Llutey/FTPServer/preferences/PreferencesFactory;<br>
.super Ljava/lang/Object;<br>
.source "PreferencesFactory.java"<br>

　　如果原java代码有经过混淆，那一般.class里面的类名和.source的源文件名会不一样，以下是经过混淆的(类名正常是xxx/PreferencesFactory，混淆以后变成xxx/d)：<br>
>.class public final Llutey/FTPServer/preferences/d;<br>
.super Ljava/lang/Object;<br>
.source "PreferencesFactor<br>

#### 3.1.2、smali文件结构
　　Smali文件与java中的类是一一对应的（每个java类对应一个smali文件），包括内部类和匿名内部类也会生成对应的smali文件（典型的比如实现某个接口的匿名内部类），所以你会看到.smali文件比.java文件更多。<br>
　　smali文件是由Dalvik指令组成的，它有自己的一套规则，它的指令都是以“.”开头，常用的指令如下：<br>

| 指令 | 说明 |
| :---: | :--- |
|.class |	包名+类名 |
|.super	| 父类类名|
|.source | 源文件名称|
|.implements | 接口实现|
|.field | 定义字段|
|.method/.end method | 方法的开始与结束|
|.locals | 方法内使用的v开口的寄存器个数|
|.prologue | 表示方法中代码的开始处|
|.line | 对应java中的行数|
|.param | 指定了方法的参数|
|.paramter | 和.paramter含义相同|
|.param | 指定了方法的参数|
|.annotation/.end annotation | 注解的开始和结束|

#### 3.1.3、接口实现
　　格式如下：<br>
>#interfaces<br>
.implements <接口名称><br>

　　举例如下：<br>
>#interfaces<br>
.implements Landroid/view/View\\$OnClickListener;<br>\\$

　　其中 #interfaces 为注释<br>

#### 3.1.4、注解
　　如果一个类中使用了注解，就会出现.annotation，格式如下:<br>
>#annotations<br>
.annotation [注解的属性] <注解类名><br>
    [注解字段=值]<br>
    ...<br>
.end annotation<br>

　　举例如下：
>#annotations<br>
.annotation build Landroid/annotation/TargetApi;<br>
    value = 0xb<br>
.end annotation<br>

　　其中 #annotations 为注释<br>

#### 3.1.5、变量定义
　　使用.field描述字段的定义，分为静态变量和普通变量两种：<br>
　　**静态变量**<br>
　　格式如下：<br>
#static fields<br>
.field <访问权限> static [修饰词] <字段名>:<变量类型><br>

　　使用# static fields注释，并加了static修饰符，举例说明：<br>
　　smali:<br>
>#static fields<br>
.field public static mStr:Ljava/lang/String;<br>

　　java:<br>
>public static final String mStr;<br>

　　**普通变量**<br>
　　格式如下：<br>
>#instance fields<br>
.field <访问权限修饰符> [非权限修饰符] <变量名>:<变量类型><br>

　　使用# instance fields注释，举例说明：<br>
　　smali:<br>
>.field private bool:Z<br>

　　java:<br>
>private boolean bool;<br>

#### 3.1.6、方法描述
  smali中的方法以.method/.end method进行描述，有分两种方法，一种是直接方法，一种是虚方法，
什么事直接方法和虚方法呢？直接方法就是不能被覆写的方法，包括用static，private修饰的方法，虚方法表示可以被覆写的方法，包括public，protected修饰的方法。<br>
　　两者在smali中的注释分别是直接方法(#direct methods)，虚方法(#virtual methods),一般直接方法在smali文件的前半部分，虚方法在后半部分。<br>
　　方法的格式如下：<br>
>#direct methods/#virtual methods<br>
.method <访问权限修饰符> [非访问权限修饰符] <方法原型><br>
　　　<.locals><br>
　　　[.parameter]<br>
　　　[.prologue]<br>
　　　[.line]<br>
　　　<代码逻辑><br>
.end<br>

　　其中.parameter,.prologue,.line是可选的。举例说明：<br>
>#virtual methods<br>
.method public final onClick(Landroid/view/View;)V<br>
　　　.locals 1<br>
<br>
　　　.prologue<br>
　　　.line 71<br>
　　　iget-object v0, p0, Llutey/FTPServer/preferences/sdk11/pre/a;->a:Llutey/FTPServer/preferences/sdk11/pre/FTPServerPreferences;<br>
<br>
　　　invoke-static {v0}, Llutey/FTPServer/preferences/sdk11/pre/FTPServerPreferences;->a(Llutey/FTPServer/preferences/sdk11/pre/FTPServerPreferences;)V<br>
<br>
　　　.line 72<br>
　　　return-void<br>
.end method<br>

　　注意几个特殊的方法：<br>
>.method xxx constructor \<clinit\>()V  //类中final变量的初始化<br>
>.method xxx constructor \<init\>()V   //类中的构造行数<br>
.method static synthetic methodName(xxxx)V <br>
//加synthetic修饰符的方法，synthetic是合成的意思，即这个方法在原java代码里面是没有的，是在java编译成Dalvik字节码的时候合成的，后续会详细分析这类方法<br>

　　方法中具体的语法在文章的后半部分详细为您说明。<br>

#### 3.1.7、smali数据类型
  Dalvik字节码和Java一样，都只有两种数据类型：基本类型和引用类型，8中基本数据类型，对象和数组是引用类型，Dalvik字节码和Jvm中对数据类型的描述是一致的。<br>
　　基本类型和无返回类型用一个大写字母表示。<br>
　　对象类型用一个大写字母L加对象的全限定名表示，如Ljava/lang/String表示字符串对象类型。<br>
　　一维数组用一个 "[" 加数据类型或对象类型表示。<br>
　　java数据类型和Dalvik字节码的数据类型一一对应，对应关系如下表：<br>

|java类型|smali描述符|
| :---: | :---:|
|boolean|Z|
|byte|B|
|short|S|
|char|C|
|int|I|
|long|L|
|float|F|
|double|D|
|void|V|
|对象类型|L|
|数组类型|[|

#### 3.1.8、基本数据类型
  Z,B,S,C,I,L,F,D为基本数据类型，从上表可以看出，Dalvik字节码基本类型的描述符基本上是java基本类型的首字母，除了boolean对应为Z外。<br>

#### 3.1.9、对象类型
  L加上类或者接口的全称表示对象类型，即Lpackage/objectName，如String类型描述符为Ljava/lang/String，包com.biyou下面的test类的类型描述符为Lcom/biyou/test。<br>

#### 3.1.10、数组类型
  基本类型的数组为 “[”加上基本类型描述符来表示，一维数组前面是一个“[”,多一个维度前面多加一个“[”,比如int类型，一维是:[I,二维是:[[I,依次类推。<br>
　　对象类型的数组为“[”加上对象类型表示符来表示，如String类型表示为:[Ljava/lang/String。<br>

#### 3.1.11、smali语法
  smali的语法跟汇编语言有点类似，但没有汇编那么复杂，可以很轻松的看懂，推荐使用source insight来看。<br>

#### 3.1.12、变量的描述
　　格式为：<br>
>对象类型描述符->变量名:类型描述符;<br>

　　举例说明：<br>
>Lcom/biyou/test;->count:I  //基本类型<br>
Lcom/biyou/test;->str:Ljava/lang/String  //对象类型<br>

　　Dalvik对变量的描述都会指明变量定义的java类和变量的类型，如例子中的count变量的描述，说明它定义在com.biyou.test类，是int类型的变量。<br>

#### 3.1.13、方法的描述
　　格式为:<br>
>对象类型描述符->方法名(参数类型描述符)返回值类型描述符;<br>

　　下面我们通过几个例子来说明,以java.lang.String为例:<br>
>java方法:public char charAt(int index){...}<br>
Davilk描述:Ljava/lang/String;->charAt(I)C<br>
<br>
java方法:public void getChars(int srcBegin,int srcEnd,char dst[],int dstBegin){...}<br>
Davilk描述:Ljava/lang/String;->getChars(II[CI)V<br>
<br>
java方法:public boolean equals(Object anObject){...}<br>
Davilk描述:Ljava/lang/String;->equals(Ljava/lang/Object)Z<br>


### 3.２、Dalvik指令格式
　　一段Dalvik汇编代码由一系列Dalvik指令组成，指令语法由指令的**位描述**与**指令格式标识**来决定。<br>
#### 3.2.1、指令的位描述
　　位描述约定如下：<br>
>1、每16位的字采用空格分隔开来。<br>
2、每个字母表示四位，每个字母按顺序从高字节开始，排列到低字节。每四位之间可能使用竖线“丨”来表示不同的内容。<br>
3、顺序采用 A〜Z 的单个大写字母作为一个4位的操作码，op 表示一个8位的操作码。<br>
4、“∅”来表示这字段所有位为0值。<br>

　　以指令格式“A|G|op BBBB F|E|D|C”为例：<br>
　　指令中间有两个空格，每个分开的部分大小为16位，所以这条指令由三个16位的字组成。第一个16位是“A|G|op”，高8位由A与G组成，低字节由操作码op组成。第二个16位由“BBBB”组成，它表示一个16位的偏移值。第三个16位分别由F、E、D、C共四个4 字节组成，在这里它们表示寄存器参数。<br>
#### 3.2.2、指令的格式标识
　　单独使用位标识还无法确定一条指令，必须通过指令格式标识来指定指令的格式编码。它的约定如下：<br>
>1、指令格式标识大多由三个字符组成，前两个是数字，最后一个是字母。<br>
2、第一个数字是表示指令有多少个16位的字组成。<br>
3、第二个数字是表示指令最多使用寄存器的个数。特殊标记“r”标识使用一定范围内的寄存器<br>
4、第三个字母为类型码，表示指令用到的额外数据的类型<br>

　　还有一种特殊情况，指令格式标识末尾可能会多出另一个字母，如果是字母“s”表示指令采用静态链接，如果是字母“i”表示指令应该被内联处理。<br>
#### 3.2.3、指令格式标识的类型码
|助记符|位大小|说明|
|:---|:---:|:---|
|b|8|8位有符号立即数|
|c|16,32|常量池索引|
|f|16|接口常量（仅对静态链接格式有效）|
|h|16|有符号立即数（32位数或64位数的高值位，低值位为0）|
|i|32|立即数，有符号整数或32位浮点数|
|l|64|立即数，有符号整数或64位双精度浮点数|
|m|16|方法常量（仅对静态链接格式有效）|
|n|4|4位的立即数|
|s|16|短整型立即数|
|t|8,16,32|跳转，分支|
|x|0|没有额外数据|

　　指令格式标识实例：22x<br>
　　第一个数字 2 表示指令有两个 16 位字组成，第二个数字 2 表示指令使用到 2 个寄存器， 第三个字母 x 表示没有使用到额外的数据。<br>

#### 3.2.4、Dalvik指令语法说明
　　Dalvik指令对语法约定如下：<br>
>1、每条指令从操作码开始，后面紧跟参数，参数个数不定，每个参数之间采用逗号分开。<br>
2、每条指令的参数从指令第一部分开始，op位于低8位，高8位可以是一个8位的 参数，也可以是两个4位的参数，还可以为空，如果指令超过16位，则后面部分依次作为参数。<br>
3、如果参数采用“vX”的方式表示，表明它是一个寄存器，如v0、v1等。这里采用 v 而不用 r 是为了避免与基于该虚拟机架构本身的寄存器命名产生冲突，如ARM 架构寄存器命名采用 r 开头。<br>
4、如果参数采用“#+X”的方式表示，表明它是一个常量数字。<br>
5、如果参数采用“+X”的方式表示，表明它是一个相对指令的地址偏移。<br>
6、如果参数采用“kind@X”的方式表示，表明它是一个常量池索引值。其中 kind 表示常量池类型，它可以是“string”（字符串常量池索引）、“type”（类型常量池索引）、“field”（字段常量池索引）或者“meth”（方法常量池索引）。<br>

　　指令实例：op vAA, string@BBBB
　　指令用到了 1个寄存器参数vAA，并且还附加了一个字符串常量池索引string@BBBB。其实这条指令格式代表着 const-string 指令。<br>

## 3.3、了解Dalvik寄存器

### 3.3.1、DEX文件反汇编工具
　　目前DEX可执行文件主流的反汇编工具有 **BakSmali** 与 **Dedexer** 。两种工具的反汇编代码大体的结构组织是一样的，在方法名、字段类型与代码指令序列上它们保持一致，具体的差异表现在一些语法细节上。<br>
　　BakSmali 提供反汇编功能的同时，还支持使用 Smali 工具打包反汇编代码重新生成 dex 文件，该功能被广泛应用于 apk 文件的修改、补丁、破解等场合，因而更加受到开发人员的青睐。<br>
### 3.3.2、Dalvik寄存器原理

### 3.3.3、寄存器表示法— v 命名法与 p 命名法
　　假设一个函数使用到M个寄存器，并且该函数有N个参数，根据 Dalvik 虚拟机参数传递方式中的规定：参数使用最后的N个寄存器，局部变量使用从v0开始的前M- N个寄存器。如前面小节中，foo() 函数使用到了5个寄存器，2个显式的整形参数，其中foo()函数是 Hello类的非静态方法，函数被调用时回传入一个隐式的Hello对象引用，因此，世纪传入的参数是3个。根据传参规则，局部变量将使用前2个寄存器，参数会使用后3个寄存器。<br>
　　**v命名法：**<br>
　　v命名方法采用小写字母“v”开头的方式表示函数中用到局部变量与参数，所有的寄存器命名从v0开始，依次递增。对于foo()函数，v命名法会用到v0,v1,v2,v3,v4等五个寄存器。v0与v1表示函数的局部变量寄存器，v2表示被传入的Hello对象的引用，v3与v4分别表示传入的整形参数。<br>
　　**p命名法**<br>
　　p命名方法对函数的局部变量的命名没有影响，它的命名规则是：函数中引入的参数从p0开始，依次递增。对于foo()函数，v命名法会用到v0,v1,p0,p1,p2等五个寄存器。v0与v1同样用来表示函数的局部变量寄存器，p0表示被传入的Hello对象的引用，p1与p2分别表示传入的整形参数。<br>
　　v命名法 与 p命名法对比表：<br>
  
  |v命名法|p命名法|寄存器含义|
  |:---:|:---:|:---|
  |v0| v0| 第一个局部变量寄存器|
  |v1| v1| 第二个局部变量寄存器|
  |...| ...| 中间的局部变量寄存器依次递增且名称相同|
  |vM-N| p0| 第一个参数寄存器|
  |...| ...| 中间的参数寄存器分别依次递增|
  |vM-1| pN-1| 第N个参数寄存器|

## 3.4、Dalvik 字节码的类型、方法与字段表示方法
　　Dalvik 字节码有着一套自己的类型、方法与字段表示方法，这些方法与Dalvik虚拟机指令集一起组成了一条条的Dalvik汇编代码。<br>
### 3.4.1、Dalvik 类型
　　Dalvik字节码只有两种类型，基本类型与引用类型。Dalvik使用这两种类型来表示Java语言的全部类型，处了对象和数组属于引用类型外，其他的Java类型都是基本类型。
　　**Dalvik 字节码类型描述符表**

|Smali类型描述符|Java类型|备注|
|:---:|:---:|:---|
|v|void|只能用于返回值类型|
|Z|boolean||
|B|byte||
|S|short||
|C|char||
|I| int||
|J| long|64位类型|
|F|float||
|D|double|64位类型|
|Lpackage/name/ObjectName;|Java 类类型|L表示是一个对象类型，后面跟着一个java类，package/name表示该对象所在的包，<br>ObjectName表示对象的名称，“;”表示对象名称结束|
|[类型|数组类型|如:[I 表示一个int型数组；[Ljava/long/String 表示一个String的对象数组|
|[[ 或 [[[| 多维数组|最大克表示为数为255个，如：[[I 表示int[][]，[[[I 表示int[][][]|

### 3.4.2、Dalvik 字段
　　字段与方法很相似，只是字段没有方法签名域中的参数与返回值，取而代之的是字段的类型。同样，Dalvik虚拟机定位字段与字节码静态分析时会用到它。字段的格式如下：<br>
>Lpackage/name/ObjectName;->FieldName:Ljava/lang/String;

　　字段由类型（Lpackage/name/ObjectName;）、字段名（FieldName）与字段类型（Ljava/lang/String;）组成。其中字段名与字段类型中间用冒号“:”隔开。<br>
　　BakSmali生成的字段代码以 .field指令开头，根据字段类型的不同，在字段指令的开始可能会用“\#”号加以注释，如：“\# instance fields”表示这是一个实例字段，“\# static fields”表示这是一个静态字段。<br>
　　静态字段的声明格式：<br>
>#static fields  <br>
.field < 访问权限> static [ 修饰关键字] < 字段名>:< 字段类型>; <br>

　　　实例字段的声明与静态字段类似，只是少了static关键字，它的格式如下:<br>
>#instance fields  <br>
.field < 访问权限> [ 修饰关键字] < 字段名>:< 字段类型>;<br>

### 3.4.3、Dalvik 方法
　　方法的表现形式比类名要复杂一些，Dalvik使用方法名、类型参数与返回值来详细描述一个方法。这样做一方面有助于Dalvik虚拟机在运行时从方法表中快速地找到正确的方法，另一方面，Dalvik虚拟机也可以使用它们来做一些静态分析，比如Dalvik字节码的验证与优化。方法格式如下：<br>
>Lpackage/name/ObjectName;->Methodname(III)Z

　　在这个例子中，Lpackage/name/ObjectName;应该理解为一个类型，MethodName为具体的方法名，(III)Z是方法的签名部分，其中括号内的III为方法的参数（在此为三个整型参数），Z表示方法的返回类型（boolean类型）。<br>
　　下面是一个更为复杂的例子：<br>
>method(I[[IILjava/lang/String;[Ljava/lang/Object;)Ljava/lang/String;

　　按照上面的知识，将其转换成Java形式的代码应该为：<br>
>String method(int, int[][], int, String, Object[])

　　BakSmali生成的方法代码 .method指令开始，以 .end method指令结束，根据方法类型的不同，在方法指令开始前可能会用“\#”号加以注释。如“\# virtual methods”表示这是一个虚方法，“\# direct methods”表示这是一个直接方法。<br>
　　直接方法格式如下：<br>
>#direct methods　　　　　　　　　　　　　　　　//添加的注释<br>
.method <访问权限> [ 修饰关键字] < 方法原型> <br> 
　　 <.locals>　　　　　　　　　　　　　　　　//指定了使用的局部变量的个数  <br>
 [.parameter]　　　　　　　　　　　　　　　　//指定了方法的参数  <br>
 [.prologue]　　　　　　　　　　　　　//指定了代码的开始处，混淆过的代码可能去掉了该指令  <br>
 [.line] 　　　　　　　　　　　　　　　　//指定了该处指令在源代码中的行号  <br>
<代码体>  <br>
.end method  <br>

　　虚方法的声明与直接方法相同，只是起始处的注释为“virtual methods”。<br>
<br>
　　如果一个类实现了接口，会在smali 文件中使用“.implements ”指令指出,相应的格式声明如下:
>#interfaces  <br>
.implements < 接口名>　　　　　　　//接口关键字  <br>
  
　　如果一个类使用了注解，会在 smali 文件中使用“.annotation ”指令指出,注解的格式声明如下:  
>#annotations  <br>
.annotation [ 注解属性] < 注解类名>  <br>
　　　[ 注解字段 =  值]  <br>
.end annotation  <br>

　　注意：注解的作用范围可以是类、方法或字段。如果注解的作用范围是类，“.annotation ”指令会直接定义在smali 文件中，如果是方法或字段，“.annotation ”指令则会包含在方法或字段定义中。
>#instance fields <br> 
.field public sayWhat:Ljava/lang/String;   <br> 
//String 类型 它使用了com.droider.anno.MyAnnoField注解，注解字段info值为“Hello my friend”<br>
　　 .annotation runtime Lcom/droider/anno/MyAnnoField;  <br>
　　　　　info = "Hello my friend"  <br>
　　 .end annotation  <br>
.end field  <br>

## 3.5、Dalvik 指令集
### 3.5.1 指令特点
　　Dalvik指定在调用格式上模仿了C语言的调用约定。Dalvik指令的语法与助词符有如下特点：<br>
>参数采用从目标（destination）到源（source）的方式。<br>
根据字节码的大小与类型不同，一些字节码添加了名称后缀以消除岐义。<br>
 32位常规类型的字节码末添加任何后缀。<br>
64位常规类型的字节码添加 -wide后缀。<br>
特殊类型的字节码根据具体类型添加后缀。它们可以是 -boolean，-byte，-char，-short，-int，-long，-float，-double，-object，-string，-class，-void之一。<br>
根据字节码的布局与选项不同，一些字节码添加了字节码后缀以消除岐义。这些后缀通过在字节码主名称后添加斜杠“/”来分隔开。<br>
在指令集的描述中，宽度值中每个字母表示宽度为4位。<br>

　　例如这条指令：“move-wide/from16vAA, vBBBB”：<br>
　　move为基础字节码（base opcode），标识这是基本操作。wide为名称后缀（name suffix），标识指令操作的数据宽度（64位）。from16为字节码后缀（opcode suffix），标识源为一个16位的寄存器引用变量。vAA为目的寄存器，它始终在源的前面，取值范围为v0~v255。vBBBB为源寄存器，取值范围为v0~v65535。<br>
　　Dalvik指令集中大多数指令用到了寄存器作为目的操作数或源操作数，其中 A/B/C/D/E/F/G/H 代表一个4位的数值，可用来表示0～15的数值或v0~v15的寄存器，而 AA/BB/CC/DD/EE/FF/GG/HH 代表一个8位的数值，可用来表示0~255的数值或v0~v255的寄存器，AAAA/BBBB/CCCC/DDDD/EEEE/FFFF/GGGG/HHHH 代表一个16位的数值，可用来表示0～65535的数值或v0~v65535的寄存器。注意：Android官方指令文档描述寄存器时，对不同取值范围的寄存器以括号说明其大小，如A:destinationregister(4 bits)，A:destination register(16 bits)。请注意，Dalvik虚拟机中的每个寄存器都是32位的。描述指令时所说的位数表示的是寄存器数值的取值范围。<br>

### 3.5.2 空操作指令
　　空操作指令的助记符为nop。它的值为00，通常nop指令被用来作对齐代码之用，无实际操作。<br>
### 3.5.3 数据操作指令
　　数据操作指令为move。move指令的原型为“move destination,source”，move指令根据字节码的大小与类型不同，后面会跟上不同的后缀。

| 指令 | 描述 |
| :--- | :--- |
|move vA, vB | 将vB寄存器的值赋给vA寄存器，源寄存器与目的寄存器都为4位。|
|move/from16 vAA, vBBBB | 将vBBBB寄存器的值赋给vAA寄存器，源寄存器为16位，目的寄存器为8位。|
|move/16 vAAAA, vBBBB | 将vBBBB寄存器的值赋给vAAAA寄存器，源寄存器与目的寄存器都为16位。|
|move-wide vA, vB | 为4位的寄存器对赋值。源寄存器与目的寄存器都为4位。|
|move-wide/from16 vAA, vBBBB <br> move-wide/16vAAAA, vBBBB | 实现与“move-wide”相同。|
|move-object vA, vB | 为对象赋值。源寄存器与目的寄存器都为4位。|
|move-object/from16 vAA, vBBBB | 为对象赋值。源寄存器为16位，目的寄存器为8位。|
|move-object/16 vAA, vBBBB | 为对象赋值。源寄存器与目的寄存器都为16位。|
|move-result vAA | 将上一个invoke类型指令操作的单字非对象结果赋给vAA寄存器。|
|move-result-wide vAA | 将上一个invoke类型指令操作的双字非对象结果赋给vAA寄存器。|
|move-result-object vAA | 将上一个invoke类型指令操作的对象结果赋给vAA寄存器。|
|move-exception vAA | 保存一个运行时发生的异常到vAA寄存器，这条指令必须是异常发生时的异常处理器的一条指令。否则的话，指令无效。|

### 3.5.4 返回指令
返回指令指的是函数结尾时运行的最后一条指令。它的基础字节码为teturn，共有以下四条返回指令：

| 指令 | 描述 |
| :--- | :--- |
|return-void | 表示函数从一个void方法返回。|
|return vAA | 表示函数返回一个32位非对象类型的值，返回值寄存器为8位的寄存器vAA。|
|return-wide vAA | 表示函数返回一个64位非对象类型的值，返回值为8位的寄存器对vAA。|
|return-object vAA | 表示函数返回一个对象类型的值。返回值为8位的寄存器vAA。|

### 3.5.5 数据定义指令
　　数据定义指令用来定义程序中用到的常量，字符串，类等数据。它的基础字节码为const。<br>

|指令　|　描述　|
|:--- | :---- |
|const/4 vA, #+B | 将数值符号扩展为32位后赋给寄存器vA。|
|const/16 vAA, #+BBBB | 将数据符号扩展为32位后赋给寄存器vAA。|
|const vAA, #+BBBBBBBB | 将数值赋给寄存器vAA。|
|const/high16 vAA, #+BBBB0000 | 将数值右边零扩展为32位后赋给寄存器vAA。|
|const-wide/16 vAA, #+BBBB | 将数值符号扩展为64位后赋给寄存器对vAA。|
|const-wide/32 vAA, #+BBBBBBBB | 将数值符号扩展为64位后赋给寄存器对vAA。|
|const-wide vAA, #+BBBBBBBBBBBBBBBB | 将数值赋给寄存器对vAA。|
|const-wide/high16 vAA, #+BBBB000000000000 | 将数值右边零扩展为64位后赋给寄存器对vAA。|
|const-string vAA, string@BBBB | 通过字符串索引构造一个字符串并赋给寄存器vAA。|
|const-string/jumbo vAA, string@BBBBBBBB | 通过字符串索引（较大）构造一个字符串并赋给寄存器vAA。|
|const-class vAA, type@BBBB | 通过类型索引获取一个类引用并赋给寄存器vAA。|
|const-class/jumbo vAAAA, type@BBBBBBBB | 通过给定的类型索引获取一个类引用并赋给寄存器vAAAA。这条指令占用两个字节，值为0xooff（Android4.0中新增的指令）。|

### 3.5.6 锁指令
　　锁指令多用在多线程程序中对同一对象的操作。Dalvik指令集中有两条锁指令：
 
|指令　|　描述　|
|:--- | :---- |
|monitor-enter vAA | 为指定的对象获取锁。|
|monitor-exit vAA | 释放指定的对象的锁。|

　　如java的com.biyou.test类需要需要加锁：<br>
　　java为：<br>
>    synchronized (test.class)<br>
    {<br>
      ...<br>
    }<br>

　　smali为:
 >    const-class v0, Lcom/biyou/test;<br>
     monitor-enter v0<br>
     ...<br>
     monitor-exit v0<br>

### 3.5.7 实例操作指令
　　与实例相关的操作包括实例的类型转换，检查及新建等：

| 指令 | 描述 |
| :--- | :---- |
|check-cast vAA, type@BBBB | 将vAA寄存器中的对象引用转换成指定的类型，如果失败会抛出ClassCastException异常。如果类型B指定的是基本类型，对于非基本类型的A来说，运行时始终会失败。|
|instance-of vA, vB, type@CCCC | 判断vB寄存器中的对象引用是否可以转换成指定的类型，如果可以vA寄存器赋值为1，否则vA寄存器赋值为0。|
|new-instance vAA, type@BBBB | 构造一个指定类型对象的新实例，并将对象引用赋值给vAA寄存器，类型符type指定的类型不能是数组类。|
|check-cast/jumbo vAAAA, type@BBBBBBBB | 功能与“check-cast vAA, type@BBBB”相同，只是寄存器值与指令的索引取值范围更大（Android4.0中新增的指令）。|
|instance-of/jumbo vAAAA, vBBBB, type@CCCCCCCC | 指令功能与“instance-of vA, vB, type@CCCC”相同，只是寄存器值与指令的索引取值范围更大（Android4.0中新增的指令）。|
|new-instance/jumbo vAAAA, type@BBBBBBBB | 指令功能与“new-instance vAA, type@BBBB”相同，只是寄存器值与指令的索引取值范围更大（Android4.0中新增的指令）。|

### 3.5.8 数组操作指令
　　数组操作包括获取数组长度，新建数组，数组赋值，数组元素取值与赋值等操作。

| 指令 | 描述 |
| :--- | :--- |
|array-length vA, vB | 获取给定vB寄存器中数组的长度并将值赋给vA寄存器，数组长度指的是数组的条目个数。|
|new-array vA, vB, type@CCCC | 构造指定类型（type@CCCC）与大小（vB）的数组，并将值赋给vA寄存器。|
|filled-new-array {vC, vD, vE, vF, vG},type@BBBB | 构造指定类型（type@BBBB）与大小（vA）的数组并填充数组内容。vA寄存器是隐含使用的，除了指定数组的大小外还指定了参数的个数，vC~vG是使用到的参数寄存序列。|
|filled-new-array/range {vCCCC  ..vNNNN}, type@BBBB | 功能与“filled-new-array {vC, vD, vE, vF, vG},type@BBBB”相同，只是参数寄存器使用range字节码后缀指定了取值范围 ，vC是第一个参数寄存器，N = A +C -1。|
|fill-array-data vAA, +BBBBBBBB | 用指定的数据来填充数组，vAA寄存器为数组引用，引用必须为基础类型的数组，在指令后面会紧跟一个数据表。|
|new-array/jumbo vAAAA,vBBBB,type@CCCCCCCC | 与“new-array vA,vB,type@CCCC”相同，只是寄存器值与指令的索引取值范围更大（Android4.0中新增的指令）。|
|filled-new-array/jumbo{vCCCC  ..vNNNN},type@BBBBBBBB | 与“filled-new-array/range{vCCCC  ..vNNNN},type@BBBB”相同，只是索引取值范围更大（Android4.0中新增的指令）。|
|arrayop vAA, vBB, vCC | 对vBB寄存器指定的数组元素进入取值与赋值。vCC寄存器指定数组元素索引，vAA寄存器用来存放读取的或需要设置的数组元素的值。读取元素使用aget类指令，元素赋值使用aput类指定，根据数组中存储的类型指令后面会紧跟不同的指令后缀，指令列表有 aget, aget-wide, aget-object, aget-boolean, aget-byte,aget-char,aget-short, aput, aput-wide, aput-object, aput-boolean, aput-byte, aput-char,aput-short。|

### 3.5.9 异常指令
　　Dalvik指令集中有一条指令用来抛出异常。

|指令　|　描述　|
|:--- | :---- |
|throw vAA | 抛出vAA寄存器中指定类型的异常。|

### 3.5.10 跳转指令
　　跳转指令用于从当前地址跳转到指定的偏移处。Dalvik指令集中有三种跳转指令：无条件跳转（goto），分支跳转（switch）与条件跳转（if）。<br>

|指令　|　描述　|
|:--- | :---- |
|goto +AA | 无条件跳转到指定偏移处，偏移量AA不能为0。|
|goto/16 +AAAA | 无条件跳转到指定偏移处，偏量AAAA不能为0。|
|goto/32 +AAAAAAAA | 无条件跳转到指定偏移处。|
|packed-switch vAA, +BBBBBBBB | 分支跳转指令。vAA寄存器为switch分支中需要判断的值，BBBBBBBB指向一个**packed-switch-payload格式**的偏移表，表中的值是**有规律递增**的偏移量。|
|sparse-switch vAA, +BBBBBBBB | 分支跳转指令。vAA寄存器为switch分支中需要判断的值，BBBBBBBB指向一个**sparse-switch-payload格式**的偏移表，表中的值是**无规律**的偏移量。|

　　**if-test vA, vB, +CCCC:**  条件跳转指令。比较vA寄存器与vB寄存器的值，如果比较结果满足就跳转到CCCC指定的偏移处。偏移量CCCC不能为0。if-test类型的指令有以下几条：

|指令　|　描述　|
|:--- | :---- |
|if-eq | 如果vA等于vB则跳转。Java语法表示为“if(vA== vB)”|
|if-ne | 如果vA不等于vB则跳转。Java语法表示为“if(vA!= vB)”|
|if-lt | 如果vA小于vB则跳转。Java语法表示为“if(vA< vB)”|
|if-ge | 如果vA大于等于vB则跳转。Java语法表示为“if(vA>= vB)”|
|if-gt | 如果vA大于vB则跳转。Java语法表示为“if(vA> vB)”|
|if-le | 如果vA小于等于vB则跳转。Java语法表示为“if(vA<= vB)”|

　　**if-testz vAA, +BBBB:** 条件跳转指令。拿vAA寄存器与0比较，如果比较结果满足或值为0时就跳转到BBBB指定的偏移处。偏移量BBBB不能为0。if-testz类型的指令有以下几条：

|指令|描述|
|:--- |:----|
|if-eqz | 如果vAA为0则跳转。Java语法表示为“if(vAA== 0)”|
|if-nez | 如果vAA不为0则跳转。Java语法表示为“if(vAA!= 0)”|
|if-ltz | 如果vAA小于0则跳转。Java语法表示为“if(vAA< 0)”|
|if-gez | 如果vAA大于等于0则跳转。Java语法表示为“if(vAA>= 0)”|
|if-gtz | 如果vAA大于0则跳转。Java语法表示为“if(vAA> 0)”|
|if-lez | 如果vAA小于等于0则跳转。Java语法表示为“if(vAA<= 0)”|

### 3.5.11 比较指令
　　比较指令用于对两个寄存器的值（浮点型或长整型）进行比较。它的格式为“cmpkind vAA, vBB, vCC”，其中vBB寄存器与vCC寄存器是需要比较的两个寄存器或寄存器对，比较的结果放到vAA寄存器。Dalvik指令集中共有5条比较指令：
 
|指令|描述|
|:--- | :---- |
｜cmpl-float　｜　比较两个单精度浮点数。如果vBB寄存器大于vCC寄存器，结果为-1，相等则结果为0，小于的话结果为1｜
｜cmpg-float　｜　比较两个单精度浮点数。如果vBB寄存器大于vCC寄存器，则结果为1，相等则结果为0，小于的话结果为-1｜
｜cmpl-double　｜　比较两个双精度浮点数。如果vBB寄存器对大于vCC寄存器对，则结果为-1，相等则结果为0，小于则结果为1｜
｜cmpg-double　｜　比较两个双精度浮点数。如果vBB寄存器对大于vCC寄存器对，则结果为1，相等则结果为0，小于的话，则结果为-1｜
｜cmp-long　｜　比较两个长整型数。如果vBB寄存器大于vCC寄存器，则结果为1，相等则结果为0，小则结果为-1｜


### 3.5.12 字段操作指令
　　字段操作指令用来对对象实例的字段进入读写操作。字段的类型可以是Java中有效的数据类型。对普通字段与静态字段操作有两种指令集，分别是“iinstanceopvA, vB, fidld@CCCC” 与 “sstaticop vAA, field@BBBB”。<br>
　　普通字段指令的指令前缀为i，如对普通字段读操作使用 iget 指令，写操作使用 iput 指令，既前缀是i的iput-type和iget-type指令用于字段的读写操作。<br>
　　静态字段的指令前缀为s，如对静态字段读操作使用 sget 指令，写操作使用 sput 指令，既前缀是s的sput-type和sget-type指令用于静态字段的读写操作。<br>
　　根据访问的字段类型不同，字段操作指令后面会紧跟字段类型的后缀，如 iget-byte指令表示读取实例字段 的值类型为字节类型，iput-short指令表示设置实例字段的值类型为短整型。两类指令操作结果都是一样，只是指令前缀与操作的字段类型不同。<br>
　　普通字段操作指令有：iget，iget-wide，iget-object，iget-boolean，iget-byte，iget-char，iget-short，iput，iput-wide，iput-object，iput-boolean，iput-byte，iput-char，iput-short。<br>

|指令 | 描述|
| :--- | :---- |
|iget-object vAA,vBB,filed_id | 读取vAA寄存器中的对象中的filed_id对象的引用值给vBB寄存器|
|iget-boolean vAA,vBB,filed_id | 读取vAA寄存器中的对象中的filed_id的值给vBB寄存器|
|iget-wide vAA,vBB,filed_id | 读取vAA寄存器中的对象中的filed_id的值给vBB寄存器|
|iget vAA,vBB,filed_id | 读取vAA寄存器中的对象中的filed_id的值给vBB寄存器|
|iput-object vAA,vBB,filed_id | 把vAA寄存器指向的对象的引用赋值给vBB寄存器中的filed_id对象|
|iput-boolean vAA,vBB,filed_id | 把vAA寄存器的值给vBB寄存器中的boolean类型|
|iput-wide vAA,vBB,filed_id | 把vAA寄存器的值给vBB寄存器中的wide类型|
|iput vAA,vBB,filed_id | 把vAA寄存器的值给vBB寄存器中的int类型|

　　静态字段操作指令有：sget，sget-wide，sget-object，sget-boolean，sget-byte，sget-char，sget-short，sput，sput-wide，sput-object，sput-boolean，sput-byte，sput-char，sput-short。<br>

|指令|描述|
|:--- |:----|
|sget-object vAA,vBB,filed_id | 读取vAA寄存器中的对象中的filed_id对象的引用值给vBB寄存器|
|sget-boolean vAA,vBB,filed_id | 读取vAA寄存器中的对象中的filed_id的值给vBB寄存器|
|sget-wide vAA,vBB,filed_id | 读取vAA寄存器中的对象中的filed_id的值给vBB寄存器|
|sget vAA,vBB,filed_id | 读取vAA寄存器中的对象中的filed_id的值给vBB寄存器|
|sput-object vAA,vBB,filed_id | 把vAA寄存器指向的对象的引用赋值给vBB寄存器中的filed_id对象|
|sput-boolean vAA,vBB,filed_id | 把vAA寄存器的值给vBB寄存器中的boolean类型|
|sput-wide vAA,vBB,filed_id | 把vAA寄存器的值给vBB寄存器中的wide类型|
|sput vAA,vBB,filed_id | 把vAA寄存器的值给vBB寄存器中的int类型|

　　在Android4.0系统中，Dalvik指令集中增加了“iinstanceop/jumbo vAAAA,vBBBB, field@CCCCCCCC”与"sstaticop/jumbo vAAAA,field@BBBBBBBB"两类指令，它们与上面介绍的两类指令作用相同，只是在指令中增加了jumbo字节码后缀，且寄存器值与指令的索引取值范围更大。<br>

### 3.5.13 方法调用指令
　　方法调用指令负责调用类实例的方法。它的基础指令为 invoke，方法调用指令有“invoke-kind {vC, vD, vE, vF,vG},meth@BBBB”与“invoke-kind/range {vCCCC  ..vNNNN},meth@BBBB”两类，两类指令在作用上并无不同，只是后者在设置参数寄存器时使用了range来指定寄存器的范围。根据方法类型的不同，共有如下五条方法调用指令：

| 指令 | 描述 |
| :--- | :---: |
|invoke-virtual 或 invoke-virtual/range | 调用实例的虚方法。|
|invoke-super 或 invoke-super/range | 调用实例的父类方法。|
|invoke-direct 或 invoke-direct/range | 调用实例的直接方法。|
|invoke-static 或 invoke-static/range | 调用实例的静态方法。|
|invoke-interface 或 invoke-interface/range | 调用实例的接口方法。|

　　在Android4.0系统中，Dalvik指令集中增加了“invoke-kind/jumbo {vCCCC .. vNNNN},meth@BBBBBBBB”这类指令，它与上面介绍的两类指令作用相同，只是在指令中增加了jumbo字节码后缀，且寄存器值与指令的索引取值范围更大。<br>
　　方法调用指令的返回值必须使用move-result\*指令来获取。如下面两条指令：
>invoke-static {}, Landroid/os/Parcel;->obtain() Landroid/os/Parcel;<br>
move-result-object v0<br>

### 3.5.14 数据转换指令
　　数据转换指令用于将一种类型的数值转换成另一种类型。它的格式为“unop vA, vB”，vB寄存器或vB寄存器对存放需要转换的数据，转换后的结果保存在vA寄存器或vA寄存器对中。

| 指令 | 描述 |
| :--- | :---: |
|neg-int | 对整型数求补。|
|not-int | 对整型数求反。|
|neg-long | 对长整型数求补。|
|not-long | 对长整型数求反。|
|neg-float | 对单精度浮点型数求补。|
|neg-double | 对双精度浮点型数求补。|
|int-to-long | 将整型数转换为长整型。|
|int-to-float | 将整型数转换为单精度浮点型数。|
|int-to-dobule | 将整型数转换为双精度浮点数。|
|long-to-int | 将长整型数转换为整型。|
|long-to-float | 将长整型数转换为单精度浮点型。|
|long-to-double | 将长整型数转换为双精度浮点型。|
|float-to-int | 将单精度浮点数转换为整型。|
|float-to-long | 将单精度浮点数转换为长整型数。|
|float-to-double | 将单精度浮点数转换为双精度浮点型数。|
|double-to-int | 将双精度浮点数转换为整型。|
|double-to-long | 将双精度浮点数转换为长整型。|
|double-to-float | 将双精度浮点数转换为单精度浮点型。|
|int-to-byte | 将整型转换为字节型。|
|int-to-char | 将整型转换为字符型。|
|int-to-short | 将整型转换为短整型。|
|
### 3.5.15 数据运算指令
　　数据运算指令包括算术运算指令与逻辑运算指令。算术运算指令主要进行数值间如加，减，乘，除，模，移位等运算。逻辑运算指令主要进行数值间与，或，非，抑或等运算。数据运算指令有如下四类（数据运算时可能是在寄存器或寄存器对间进行，下面的指令作用讲解时使用寄存器来描述）：

| 指令 | 描述 |
| :--- | :---: |
|binop vAA, vBB, vCC | 将vBB寄存器与vCC寄存器进行运算，结果保存到vAA寄存器。|
|binop/2addr vA, vB | 将vA寄存器与vB寄存器进行运算，结果保存到vA寄存器。|
|binop/lit16 vA, vB, #+CCCC | 将vB寄存器与常量 CCCC进行运算，结果保存到vA寄存器。|
|binop/lit8 vAA, vBB, #+CC | 将vBB寄存器与常量CC进行运算，结果保存到vAA寄存器。|

　　后面3类指令比第1类指令分别多出了2addr，lit16，lit8等指令后缀。四类指令中基础字节码相同的指令执行的运算操作是类似的，第1类指令中，根据数据的类型不同会在基础字节码后面加上数据类型后缀，如 -int 或 -long 分别表示操作的数据类型为整型与长整型。第1类指令可归类如下：

| 指令 | 描述 |
| :--- | :---: |
|add-type | vBB寄存器与vCC寄存器值进行加法运算（vBB + vCC）|
|sub-type | vBB寄存器与vCC寄存器值进行减法运算（vBB - vCC）|
|mul-type | vBB寄存器与vCC寄存器值进行乘法运算（vBB * vCC）|
|div-type | vBB寄存器与vCC寄存器值进行除法运算（vBB / vCC）|
|rem-type | vBB寄存器与vCC寄存器值进行模运算（vBB % vCC）|
|and-type | vBB寄存器与vCC寄存器值进行与运算（vBB & vCC）|
|or-type | vBB寄存器与vCC寄存器值进行或运算（vBB | vCC）|
|xor-type | vBB寄存器与vCC寄存器值进行异或运算（vBB ^ vCC）|
|shl-type | vBB寄存器值（有符号数）左移vCC位（vBB << vCC ）|
|shr-type | vBB寄存器值（有符号）右移vCC位（vBB >> vCC）|
|ushr-type | vBB寄存器值（无符号数）右移vCC位（vBB >>> vCC）|

　　其中基础字节码后面的-type可以是-int，-long， -float，-double。后面3类指令与之类似。 

　　至此，Dalvik虚拟机支持的所有指令就介绍完了。在android4.0系统以前，每个指令的字节码只占用一个字节，范围是0x0~0x0ff。在android4.0系统中，又扩充了一部分指令，这些指令被称为扩展指令，主要是在指令助记符后添加了jumbo后缀，增加了寄存器与常量的取值范围。


## 3.6、Dalvik 指令集练习：写Dalvik版Hello Word
### 3.6.1 编写smali文件
### 3.6.2 编译smali文件
### 3.6.3 测试运行










